- Check for Colab Env
> if we are in colab env we need to get the data + scripts from github or somewhere

In [139]:
import os
if "COLAB_GPU" in os.environ:
    _useMultiProcessing = False
else:
    _useMultiProcessing = True

- enable module autoreload and fix an import bug

In [134]:
%load_ext autoreload
%autoreload 2

import sys,os
sys.path.append(os.getcwd() + '/scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- import the scripts and data

In [141]:
from scripts import data, model
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

- create the model and compile it

In [211]:
unet = model.get_do_unet()

unet.compile(optimizer="adam",
             loss="binary_crossentropy",
             loss_weights=[0.3, 0.7],
             metrics=["acc"])

unet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 188, 188, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_30 (Conv2D)             (None, 186, 186, 96  2688        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 186, 186, 96  384        ['conv2d_30[0][0]']              
 ormalization)                  )                                                           

- load the dataset

In [137]:
train_img_files = glob.glob('data/train/*.jpg')
test_img_files = glob.glob('data/test/*.jpg')

train = model.generate_train_dataset(train_img_files)

train_generator = model.generate_train_dataset(train_img_files)

train_set = model.generate_train_dataset_tf(train_img_files)

test_img_chips, test_mask_chips, test_edge_chips = model.generate_test_dataset(test_img_files)

test_set = tf.data.Dataset.from_tensor_slices((test_img_chips, (test_mask_chips, test_edge_chips)))


2022-04-24 18:40:47.431671: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 552214656 exceeds 10% of free system memory.


- slice each image to small samples of size 188*188
- imgs -> rgb image
- mask -> mask
- edge -> edge_mask

In [ ]:
train_img_chips, (train_mask_chips, train_edge_chips) =  next(train_generator)

ind = 200
fig = plt.figure(figsize=(15, 10), dpi=80)
fig.subplots_adjust(hspace=0.1, wspace=0.1)
ax = fig.add_subplot(2, 3, 1)
ax.imshow(train_img_chips)
ax = fig.add_subplot(2, 3, 2)
ax.imshow(train_mask_chips)
ax = fig.add_subplot(2, 3, 3)
ax.imshow(train_edge_chips)

- Train the model

In [210]:
_batch_size = 8
_epochs = 4
_workers = 8
_model_name = 'A'

train_steps = len(train_img_files)//_batch_size
validation_steps = len(test_img_files)//_batch_size

unet.fit(train_set.batch(_batch_size), 
         epochs=_epochs, 
         validation_data=test_set.batch(_batch_size), 
         steps_per_epoch=train_steps,
         max_queue_size=2*_workers,
         use_multiprocessing=_useMultiProcessing,
         workers=_workers,
         verbose=1,
         callbacks=model.get_callbacks(_model_name))

Epoch 1/4
1/1 [==============================] - 35s 35s/step - loss: 1.0810 - mask_loss: 0.5070 - edge_loss: 0.5739 - mask_acc: 0.4307 - edge_acc: 0.0598 - val_loss: 1.3528 - val_mask_loss: 0.6716 - val_edge_loss: 0.6812 - val_mask_acc: 0.4401 - val_edge_acc: 0.0725
Epoch 2/4
1/1 [==============================] - 35s 35s/step - loss: 1.0026 - mask_loss: 0.4514 - edge_loss: 0.5512 - mask_acc: 0.3538 - edge_acc: 0.0551 - val_loss: 1.3437 - val_mask_loss: 0.6661 - val_edge_loss: 0.6776 - val_mask_acc: 0.4401 - val_edge_acc: 0.0725
Epoch 3/4
1/1 [==============================] - 34s 34s/step - loss: 0.9466 - mask_loss: 0.4106 - edge_loss: 0.5360 - mask_acc: 0.2153 - edge_acc: 0.0437 - val_loss: 1.3360 - val_mask_loss: 0.6617 - val_edge_loss: 0.6743 - val_mask_acc: 0.4401 - val_edge_acc: 0.0725
Epoch 4/4
1/1 [==============================] - 39s 39s/step - loss: 0.8919 - mask_loss: 0.3583 - edge_loss: 0.5336 - mask_acc: 0.4324 - edge_acc: 0.0703 - val_loss: 1.3262 - val_mask_loss: 0.655

In [209]:
img = test_img_chips[np.random.randint(0, len(test_img_chips), 1)[0]]

img = np.array([np.squeeze(img)])


prediction = unet.predict(img)

np.array(prediction).min()


1.0

In [168]:
np.random.randint(0, 5, 1)[0]

1